## DV2599 Project
Group 8


***
Viktor Fransson

vifr22@student.bth.se

***

Tobias Gustafsson

togu22@student.bth.se
***

#### Initialize

In [ ]:
# Import packages
import pandas as pd

# Load dataset
train_df = pd.read_csv('playground-series-s4e11/train.csv')
display(train_df)